In [1]:
import re, io, math, textwrap
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
from datetime import datetime
from dateutil import parser as dtparse

import requests
from bs4 import BeautifulSoup
try:
    from readability import Document
except ModuleNotFoundError:
    from readability.readability import Document
import trafilatura
from pdfminer.high_level import extract_text as pdf_extract_text

SAFE_UA = "noise-to-signal/0.1 (+https://github.com/ChillCaleb/noise-to-signal)"
MAX_HTML_BYTES = 2_000_000
MAX_PDF_BYTES  = 10_000_000
CONNECT_TO = 4
READ_TO    = 12

TRACKING_KEYS_PREFIX = ("utm_",)
TRACKING_KEYS_EXACT = {"gclid","fbclid","igshid","mc_cid","mc_eid","ref","spm","yclid","ocid","cmpid","campaign","adid"}


In [2]:
def sanitize_url(raw: str) -> str:
    u = urlparse(raw.strip())
    scheme = "https" if u.scheme in ("http","https") else "https"
    kept = []
    for k, v in parse_qsl(u.query, keep_blank_values=True):
        if k in TRACKING_KEYS_EXACT or any(k.startswith(p) for p in TRACKING_KEYS_PREFIX):
            continue
        kept.append((k, v))
    query = urlencode(kept, doseq=True)
    return urlunparse((scheme, u.hostname or "", u.path or "/", "", query, ""))

def _soup(html: str) -> BeautifulSoup:
    try:
        return BeautifulSoup(html, "lxml")
    except Exception:
        return BeautifulSoup(html, "html.parser")

def pick_first(*vals):
    for v in vals:
        if v:
            return v
    return None

from typing import Optional

def guess_language_from_html(html: str) -> Optional[str]:

    m = re.search(r'<html[^>]*\blang=["\']?([a-zA-Z-]+)', html, flags=re.I)
    return m.group(1).lower() if m else None

_SENT_END = re.compile(r'(?<=[.!?])\s+(?=[A-Z0-9“"])')
def split_sentences(text: str) -> list[str]:
    text = re.sub(r'\s+', ' ', (text or '').strip())
    if not text:
        return []
    parts = _SENT_END.split(text)
    # guard against huge blobs
    return [s.strip() for s in parts if s.strip()][:2000]


In [3]:
def fetch_url(url: str):
    s_url = sanitize_url(url)
    headers = {"User-Agent": SAFE_UA, "Accept": "*/*"}
    with requests.get(s_url, headers=headers, timeout=(CONNECT_TO, READ_TO), allow_redirects=True, stream=True) as r:
        r.raise_for_status()
        ctype = (r.headers.get("Content-Type") or "").lower()
        cap = MAX_PDF_BYTES if "pdf" in ctype or r.url.lower().endswith(".pdf") else MAX_HTML_BYTES
        chunks, size = [], 0
        for chunk in r.iter_content(16384):
            if not chunk: break
            size += len(chunk)
            if size > cap: break
            chunks.append(chunk)
        return r.url, b"".join(chunks), r.headers

def extract_from_html(raw_html: bytes, final_url: str) -> dict:
    html = raw_html.decode("utf-8", errors="ignore")
    soup = _soup(html)

    og_title = soup.find("meta", property="og:title")
    og_desc  = soup.find("meta", property="og:description")
    meta_desc = soup.find("meta", attrs={"name":"description"})
    meta_time = soup.find("meta", property="article:published_time")
    time_tag  = soup.find("time")

    title = pick_first(og_title and og_title.get("content"), soup.find("title").text.strip() if soup.find("title") else None)
    desc  = pick_first(meta_desc and meta_desc.get("content"), og_desc and og_desc.get("content"))
    published_raw = pick_first(meta_time and meta_time.get("content"), time_tag and time_tag.get("datetime"))

    canon = soup.find("link", rel=lambda x: x and "canonical" in x)
    canonical_url = sanitize_url(canon.get("href")) if canon and canon.get("href") else sanitize_url(final_url)

    # main text: trafilatura → readability fallback
    main_text = trafilatura.extract(html, include_comments=False, include_tables=False)
    if not main_text:
        try:
            doc = Document(html)
            main_text = _soup(doc.summary()).get_text("\n").strip()
        except Exception:
            main_text = None

    record = {
        "canonical_url": canonical_url,
        "content_type": "text/html",
        "title": title,
        "description": (desc or None),
        "published_ts": (dtparse.parse(published_raw).isoformat() if published_raw else None),
        "language": (guess_language_from_html(html) or "en"),
        "text": (main_text or None),
        "raw_len": len(html),
    }
    record["paywall_suspect"] = is_likely_paywalled(record, html)
    return record

def extract_from_pdf(raw_pdf: bytes, final_url: str) -> dict:
    try:
        txt = pdf_extract_text(io.BytesIO(raw_pdf))
        txt = txt.strip() if txt else None
    except Exception:
        txt = None
    return {
        "canonical_url": sanitize_url(final_url),
        "content_type": "application/pdf",
        "title": None,
        "description": None,
        "published_ts": None,
        "language": "en",
        "text": txt,
        "raw_len": len(raw_pdf),
        "paywall_suspect": False,
    }

# Heuristic paywall detector (lightweight & conservative)
PAYWALL_TOKENS = {
    "subscribe", "subscription", "log in", "login", "account", "unlimited access",
    "continue reading", "you have reached your limit", "gift article", "subscriber-only",
    "metered", "metering", "paywall", "please enable javascript"
}
def is_likely_paywalled(doc: dict, raw_html_text: str) -> bool:
    # Very short text after extraction + typical paywall strings in the HTML
    text_len = (len(doc.get("text") or "") if doc else 0)
    token_hits = sum(1 for t in PAYWALL_TOKENS if t in (raw_html_text or "").lower())
    # Treat NYT as stricter: if domain contains nytimes and text is thin, flag.
    domain = urlparse(doc.get("canonical_url") or "").hostname or ""
    nyt_bias = ("nytimes.com" in domain)
    return (text_len < 1200 and token_hits >= 1) or (nyt_bias and text_len < 1500)


In [4]:
from typing import Optional, List, Dict, Any

def scan_phrases(text: Optional[str], terms: List[str]) -> Dict[str, Any]:
    if not text:
        return {"hits": [], "coverage": 0.0, "n_terms": len(terms)}
    hay = text.lower()
    hits, found = [], set()
    for term in terms:
        t = term.lower().strip()
        if not t: continue
        for m in re.finditer(re.escape(t), hay):
            start, end = m.start(), m.end()
            ctx = hay[max(0, start-80): min(len(hay), end+80)]
            hits.append({"term": term, "pos": start, "context": ctx})
            found.add(term)
    denom = max(1, len([t for t in terms if t.strip()]))
    return {"hits": hits, "coverage": len(found)/denom, "n_terms": len(terms)}

# Very simple extractive summary: rank sentences by TF of non-stopwords
STOP = set("""
a an the and or but if because as while of in on to from for with without within into about over under
is are was were be been being it this that these those there here who whom whose which when where why how
by at we you they he she i me him her them us our your their its not no nor so than then too very just
""".split())

def summarize_text(text: str, max_sentences: int = 5) -> list[str]:
    sents = split_sentences(text)
    if not sents:
        return []
    # build term frequencies
    tf = {}
    for s in sents:
        for w in re.findall(r"[A-Za-z]{3,}", s.lower()):
            if w in STOP: 
                continue
            tf[w] = tf.get(w, 0) + 1
    if not tf:
        return sents[:max_sentences]
    # score sentences by sum of tf of their words
    scores = []
    for idx, s in enumerate(sents):
        score = sum(tf.get(w, 0) for w in re.findall(r"[A-Za-z]{3,}", s.lower()) if w not in STOP)
        scores.append((score, idx, s))
    scores.sort(reverse=True)
    # keep top non-duplicate sentences in original order
    chosen = sorted(scores[: max_sentences * 2], key=lambda x: x[1])  # overselect a bit, then cut
    result = [s for _, _, s in chosen][:max_sentences]
    return result


In [5]:
from typing import Optional, List, Dict, Any
import os

ALLOWED_NEWS_DOMAINS = {
    "reuters.com", "apnews.com", "cnbc.com",
    "imf.org", "whitehouse.gov", "federalreserve.gov", "treasury.gov",
    "ecb.europa.eu", "oecd.org", "un.org"
}

def domain_amp_variant(url: str) -> Optional[str]:
    """
    Common AMP patterns for big outlets.
    Returns an AMP URL string or None if no obvious variant.
    """
    u = urlparse(url)
    host = (u.hostname or "").lower()

    # NYT
    if "nytimes.com" in host:
        # preserve query if exists
        q = u.query
        if "outputType=amp" not in q:
            q = (q + "&" if q else "") + "outputType=amp"
        return urlunparse((u.scheme or "https", host, u.path, u.params, q, u.fragment))

    # Washington Post
    if "washingtonpost.com" in host:
        path = u.path
        if not path.endswith("/amp"):
            path = path.rstrip("/") + "/amp"
        return urlunparse((u.scheme or "https", host, path, u.params, u.query, u.fragment))

    # The Guardian (example)
    if "theguardian.com" in host:
        path = u.path
        if not path.endswith("/amp"):
            path = path.rstrip("/") + "/amp"
        return urlunparse((u.scheme or "https", host, path, u.params, u.query, u.fragment))

    return None

def slug_to_query(url: str, max_terms: int = 8) -> str:
    """
    Convert URL path into a compact NewsAPI query (drop dates/stopwords).
    """
    stop = set("the a an and or of to in on for with from at by as about into over under after before".split())
    path = (urlparse(url).path or "").strip("/")
    # split on slashes and hyphens, keep alphabetic tokens
    parts = re.split(r"[/-]+", path)
    words = []
    for p in parts:
        if len(p) < 3: 
            continue
        if re.fullmatch(r"\d{2,4}", p):  # likely a year/day segment
            continue
        w = re.sub(r"[^A-Za-z]", "", p).lower()
        if len(w) >= 3 and w not in stop:
            words.append(w)
    # dedupe but keep order
    seen = set(); terms = []
    for w in words:
        if w not in seen:
            terms.append(w); seen.add(w)
    return " ".join(terms[:max_terms]) or path.replace("/", " ")

def fetch_related_from_newsapi(query: str, page_size: int = 5) -> List[Dict[str, Any]]:
    """
    Query NewsAPI for related coverage and return lightweight items.
    Filters to friendlier domains when possible.
    """
    key = os.getenv("NEWSAPI_KEY")
    if not key:
        return []

    base = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "language": "en",
        "sortBy": "publishedAt",
        "pageSize": page_size,
        "apiKey": key,
    }
    try:
        r = requests.get(base, params=params, timeout=(CONNECT_TO, READ_TO))
        r.raise_for_status()
        items = (r.json() or {}).get("articles", []) or []

        # Prefer allowed domains first
        def host(u):
            try: return (urlparse(u).hostname or "").lower()
            except: return ""
        allowed = [x for x in items if host(x.get("url") or "") in ALLOWED_NEWS_DOMAINS]
        chosen = allowed or items

        out = []
        for x in chosen[:page_size]:
            out.append({
                "source": (x.get("source") or {}).get("name"),
                "title": x.get("title"),
                "url": x.get("url"),
                "publishedAt": x.get("publishedAt"),
                "description": x.get("description"),
            })
        return out
    except Exception:
        return []


In [6]:
def open_and_read(url: str) -> dict:
    try:
        # 1️⃣  Fetch and extract normally
        blob, final_url, domain, is_agg = fetch_url(url)
        doc = extract_from_html(blob, final_url)
        doc.update({...})  # metadata, timestamps, etc.
        return doc

    except requests.exceptions.RequestException as e:
        # 2️⃣  If we get a 403 or timeout
        if "403" in str(e) or "Forbidden" in str(e) or "timed out" in str(e):
            
            # (a) AMP fallback: look for AMP link in HTML (if we have partial content)
            amp_url = find_amp_url(blob)
            if amp_url:
                try:
                    blob, final_url, domain, is_agg = fetch_url(amp_url)
                    doc = extract_from_html(blob, final_url)
                    doc["fetched_via"] = "AMP"
                    return doc
                except Exception:
                    pass

            # (b) NewsAPI fallback: search related coverage
            query_terms = slug_to_query(url)
            related = fetch_related_from_newsapi(query_terms)
            return {
                "blocked": True,
                "source_url": url,
                "related_coverage": related,
                "summary_hint": "Original article paywalled; using related coverage."
            }

        # (c) Other network errors
        return {"error": str(e), "blocked": False, "source_url": url}

    except Exception as e:
        # (d) Anything else
        return {"error": f"Unhandled: {e}", "source_url": url}


In [7]:
def open_and_read(url: str) -> Dict[str, Any]:
    """
    Fetch + extract with fallbacks:
      1) normal fetch
      2) AMP variant (if applicable) on 403/timeout
      3) NewsAPI related coverage (structured) if still blocked
    """
    try:
        final_url, blob, headers = fetch_url(url)
        ctype = (headers.get("Content-Type") or "").lower()
        domain = (urlparse(final_url).hostname or "").lower()

        if "pdf" in ctype or final_url.lower().endswith(".pdf"):
            doc = extract_from_pdf(blob, final_url)
        else:
            doc = extract_from_html(blob, final_url)

        doc.update({
            "domain": domain,
            "bytes": len(blob),
            "fetched_at": datetime.utcnow().isoformat(timespec="seconds"),
            "blocked": False,
            "fetched_via": "direct",
        })
        # add preview
        if doc.get("text"):
            lines = [ln.strip() for ln in doc["text"].splitlines() if ln.strip()]
            preview = " ".join(lines)
            doc["summary_hint"] = (preview[:600] + "…") if len(preview) > 600 else preview
        else:
            doc["summary_hint"] = doc.get("description")
        return doc

    except requests.exceptions.RequestException as e:
        msg = str(e).lower()
        is_forbidden = ("403" in msg) or ("forbidden" in msg)
        is_timeout   = ("timed out" in msg) or ("timeout" in msg)

        if is_forbidden or is_timeout:
            # 2) AMP variant try
            amp = domain_amp_variant(url)
            if amp:
                try:
                    final_url, blob, headers = fetch_url(amp)
                    ctype = (headers.get("Content-Type") or "").lower()
                    domain = (urlparse(final_url).hostname or "").lower()
                    if "pdf" in ctype or final_url.lower().endswith(".pdf"):
                        doc = extract_from_pdf(blob, final_url)
                    else:
                        doc = extract_from_html(blob, final_url)
                    doc.update({
                        "domain": domain,
                        "bytes": len(blob),
                        "fetched_at": datetime.utcnow().isoformat(timespec="seconds"),
                        "blocked": False,
                        "fetched_via": "amp",
                    })
                    if doc.get("text"):
                        lines = [ln.strip() for ln in doc["text"].splitlines() if ln.strip()]
                        preview = " ".join(lines)
                        doc["summary_hint"] = (preview[:600] + "…") if len(preview) > 600 else preview
                    else:
                        doc["summary_hint"] = doc.get("description")
                    return doc
                except Exception:
                    pass

            # 3) Related coverage via NewsAPI
            q = slug_to_query(url)
            related = fetch_related_from_newsapi(q, page_size=5)
            return {
                "canonical_url": sanitize_url(url),
                "content_type": "text/html",
                "title": None, "description": None,
                "published_ts": None, "language": "en",
                "text": None, "summary_hint": "Original article blocked; showing related coverage.",
                "domain": (urlparse(url).hostname or "").lower(),
                "bytes": 0,
                "fetched_at": datetime.utcnow().isoformat(timespec="seconds"),
                "blocked": True,
                "fetched_via": "newsapi_related",
                "related_coverage": related,
                "paywall_suspect": True,
            }

        # Other network errors
        return {
            "canonical_url": sanitize_url(url),
            "content_type": None,
            "title": None, "description": None,
            "published_ts": None, "language": "en",
            "text": None, "summary_hint": None,
            "domain": (urlparse(url).hostname or "").lower(),
            "bytes": 0, "fetched_at": datetime.utcnow().isoformat(timespec="seconds"),
            "blocked": False, "fetched_via": "error",
            "error": str(e), "paywall_suspect": False,
        }

    except Exception as e:
        return {
            "canonical_url": sanitize_url(url),
            "content_type": None,
            "title": None, "description": None,
            "published_ts": None, "language": "en",
            "text": None, "summary_hint": None,
            "domain": (urlparse(url).hostname or "").lower(),
            "bytes": 0, "fetched_at": datetime.utcnow().isoformat(timespec="seconds"),
            "blocked": False, "fetched_via": "exception",
            "error": str(e), "paywall_suspect": False,
        }


In [8]:
import requests
from requests.adapters import HTTPAdapter, Retry

def fetch_url(url: str):
    # Browser-y headers so basic bot filters chill
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/123.0.0.0 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
        "Referer": "https://www.google.com/",
        "DNT": "1",
    }

    s = requests.Session()
    s.headers.update(headers)

    # Robust retries for transient 403/5xx/connection resets
    retries = Retry(
        total=3,
        backoff_factor=0.6,
        status_forcelist=(403, 408, 429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET", "HEAD"])
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.mount("http://",  HTTPAdapter(max_retries=retries))

    r = s.get(url, timeout=(CONNECT_TO, READ_TO), allow_redirects=True)
    r.raise_for_status()

    # normalize content-type
    ctype = (r.headers.get("Content-Type") or "").lower()
    return r.url, r.content, {"Content-Type": ctype}


In [ ]:
# --- TEST URL INGESTION PIPELINE (FEDERAL RESERVE PRESS RELEASE) ---

test_url = "https://www.federalreserve.gov/newsevents/pressreleases/monetary20250917a.htm"

print("🔍 Testing open_and_read() on:", test_url, "\n")

try:
    doc = open_and_read(test_url)

    print_record(
        doc,
        terms=["Federal Reserve", "rates", "inflation", "monetary", "policy", "interest", "markets"],
        summarize=True
    )

    print("\n--- STRUCTURAL CHECK ---")
    print("Blocked:", doc.get("blocked"))
    print("Fetched via:", doc.get("fetched_via"))
    print("Content type:", doc.get("content_type"))
    print("Bytes:", doc.get("bytes"))
    print("Language:", doc.get("language"))
    print("Title:", doc.get("title") or "(missing)")
    print("Text length:", len(doc.get("text") or ""))

    if doc.get("text"):
        print("\n--- TEXT PREVIEW ---")
        print(doc["text"][:400], "...")
    else:
        print("\n⚠️ No text body extracted — may be empty or malformed HTML.")

except Exception as e:
    print("❌ Test failed:", str(e))



🔍 Testing open_and_read() on: https://www.federalreserve.gov/newsevents/pressreleases/monetary20250917a.htm 

❌ Test failed: name 'print_record' is not defined


In [10]:
# Restore the built-in input function
del input


NameError: name 'input' is not defined

In [11]:
# Ask the user for a link
user_url = input("🔗 Drop link here: ")

# Import and run ingestion
from url_ingest import open_and_read

result = open_and_read(user_url)

# Optionally show a short preview
print("\n--- RESULT PREVIEW ---")
print(result.get("text", "")[:500])



KeyboardInterrupt: Interrupted by user